# Transfer learning with Huggingface using CodeFlare

In this notebook you will learn how to leverage the **[huggingface](https://huggingface.co/)** support in ray ecosystem to carry out a text classification task using transfer learning. We will be referencing the example **[here](https://huggingface.co/docs/transformers/tasks/sequence_classification)**

The example carries out a text classification task on **[imdb dataset](https://huggingface.co/datasets/imdb)** and tries to classify the movie reviews as positive or negative. Huggingface library provides an easy way to build a model and the dataset to carry out this classification task. In this case we will be using **distilbert-base-uncased** model which is a **BERT** based model.

Huggingface has a **[built in support for ray ecosystem](https://docs.ray.io/en/releases-1.13.0/_modules/ray/ml/train/integrations/huggingface/huggingface_trainer.html)** which allows the huggingface trainer to scale on CodeFlare and can scale the training as we add additional gpus and can run distributed training across multiple GPUs that will help scale out the training.


### Getting all the requirements in place

In [1]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [2]:
# Create authentication object for oc user permissions and login
auth = TokenAuthentication(
    token = "sha256~wTEk7b6J0jRiIGCCCl8f_uVRimPYqMjDjthEsQE5i9s",
    server = "https://api.mini2.mydomain.com:6443",
    skip_tls = True
)
auth.login()

'WARNING: Using insecure TLS client config. Setting this option is not supported!\n\nLogged into "https://api.mini2.mydomain.com:6443" as "kube:admin" using the token provided.\n\nYou have access to 105 projects, the list has been suppressed. You can list all projects with \'oc projects\'\n\nUsing project "huggingface".\nWelcome! See \'oc help\' to get started.\n'

Here, we want to define our cluster by specifying the resources we require for our batch workload. Below, we define our cluster object (which generates a corresponding AppWrapper).

In [3]:
# Create our cluster and submit appwrapper
cluster = Cluster(ClusterConfiguration(name='hfgputest', min_worker=1, max_worker=3, min_cpus=8, max_cpus=8, min_memory=16, max_memory=16, gpu=1, instascale=False))

Written to: hfgputest.yaml


Next, we want to bring our cluster up, so we call the `up()` function below to submit our cluster AppWrapper yaml onto the MCAD queue, and begin the process of obtaining our resource cluster.

In [4]:
cluster.up()

Now, we want to check on the initial status of our resource cluster, then wait until it is finally ready for use.

In [5]:
cluster.status()

╭─────────────────────────╮
│     🚀 Cluster Queue    │
│        Status 🚀        │
│ +-----------+---------+ │
│ | Name      | Status  | │
│ +===========+=========+ │
│ | hfgputest | pending | │
│ |           |         | │
│ +-----------+---------+ │
╰─────────────────────────╯

(<CodeFlareClusterStatus.QUEUED: 3>, False)

In [6]:
cluster.wait_ready()

Waiting for requested resources to be set up...
Requested cluster up and running!


In [7]:
cluster.status()

                     🚀 CodeFlare Cluster Status 🚀                     
                                                                        
 ╭────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                             │ 
 │   hfgputest                                            Active ✅   │ 
 │                                                                    │ 
 │   URI: ray://hfgputest-head-svc.huggingface.svc:10001              │ 
 │                                                                    │ 
 │   ]8;id=701788;ray-dashboard-hfgputest-huggingface.apps.mini2.mydomain.com\Dashboard🔗]8;;\                                                      │ 
 │                                                                    │ 
 ╰────────────────────────────────────────────────────────────────────╯

(<CodeFlareClusterStatus.READY: 1>, True)

Let's quickly verify that the specs of the cluster are as expected.

In [8]:
cluster.details()

                     🚀 CodeFlare Cluster Details 🚀                    
                                                                        
 ╭────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                             │ 
 │   hfgputest                                            Active ✅   │ 
 │                                                                    │ 
 │   URI: ray://hfgputest-head-svc.huggingface.svc:10001              │ 
 │                                                                    │ 
 │   ]8;id=586480;http://ray-dashboard-hfgputest-huggingface.apps.mini2.mydomain.com\Dashboard🔗]8;;\                                                      │ 
 │                                                                    │ 
 │                      Cluster Resources                             │ 
 │   ╭─ Workers ──╮  ╭───────── Worker specs(each) ─────────╮         │ 
 │   │  Min  Max  │  │  Memory      CPU         GPU         │         │ 
 │   │            │  │                                      │         │ 
 │   │  1    3    │  │  16~16       8           1           │         │ 
 │   │            │  │                                      │         │ 
 │   ╰────────────╯  ╰──────────────────────────────────────╯         │ 
 ╰────────────────────────────────────────────────────────────────────╯

RayCluster(name='hfgputest', status=<RayClusterStatus.READY: 'ready'>, min_workers=1, max_workers=3, worker_mem_min=16, worker_mem_max=16, worker_cpu=8, worker_gpu=1, namespace='huggingface', dashboard='http://ray-dashboard-hfgputest-huggingface.apps.mini2.mydomain.com')

In [9]:
ray_cluster_uri = cluster.cluster_uri()
print(ray_cluster_uri)

ray://hfgputest-head-svc.huggingface.svc:10001


**NOTE**: Now we have our resource cluster with the desired GPUs, so we can interact with it to train the HuggingFace model.

In [10]:
#before proceeding make sure the cluster exists and the uri is not empty
assert ray_cluster_uri, "Ray cluster needs to be started and set before proceeding"

import ray
from ray.air.config import ScalingConfig

# reset the ray context in case there's already one. 
ray.shutdown()
# establish connection to ray cluster

#install additionall libraries that will be required for this training
runtime_env = {"pip": ["transformers", "datasets", "evaluate", "pyarrow<7.0.0"]}

ray.init(address=f'{ray_cluster_uri}', runtime_env=runtime_env)

print("Ray cluster is up and running: ", ray.is_initialized())

Ray cluster is up and running:  True


**NOTE** : in this case since we are running a task for which we need additional pip packages. we can install those by passing them in the `runtime_env` variable

### Transfer learning code from huggingface

We are using the code based on the example **[here](https://huggingface.co/docs/transformers/tasks/sequence_classification)** . 

In [11]:
@ray.remote
def train_fn():
    from datasets import load_dataset
    import transformers
    from transformers import AutoTokenizer, TrainingArguments
    from transformers import AutoModelForSequenceClassification
    import numpy as np
    from datasets import load_metric
    import ray
    from ray import tune
    from ray.train.huggingface import HuggingFaceTrainer

    dataset = load_dataset("imdb")
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    #using a fraction of dataset but you can run with the full dataset
    small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
    small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

    print(f"len of train {small_train_dataset} and test {small_eval_dataset}")

    ray_train_ds = ray.data.from_huggingface(small_train_dataset)
    ray_evaluation_ds = ray.data.from_huggingface(small_eval_dataset)

    def compute_metrics(eval_pred):
        metric = load_metric("accuracy")
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    def trainer_init_per_worker(train_dataset, eval_dataset, **config):
        model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

        training_args = TrainingArguments("/tmp/hf_imdb/test", eval_steps=1, disable_tqdm=True, 
                                          num_train_epochs=1, skip_memory_metrics=True,
                                          learning_rate=2e-5,
                                          per_device_train_batch_size=16,
                                          per_device_eval_batch_size=16,                                
                                          weight_decay=0.01,)
        return transformers.Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            compute_metrics=compute_metrics
        )

    scaling_config = ScalingConfig(num_workers=3, use_gpu=True) #num workers is the number of gpus

    # we are using the ray native HuggingFaceTrainer, but you can swap out to use non ray Huggingface Trainer. Both have the same method signature. 
    # the ray native HFTrainer has built in support for scaling to multiple GPUs
    trainer = HuggingFaceTrainer(
        trainer_init_per_worker=trainer_init_per_worker,
        scaling_config=scaling_config,
        datasets={"train": ray_train_ds, "evaluation": ray_evaluation_ds},
    )
    result = trainer.fit()
    print(f"metrics: {result.metrics}")
    print(f"checkpoint: {result.checkpoint}")
    print(f"log_dir: {result.log_dir}")
    return result.checkpoint
    #return result.log_dir

**NOTE:** This code will produce a lot of output and will run for **approximately 2 minutes.** As a part of execution it will download the `imdb` dataset, `distilbert-base-uncased` model and then will start transfer learning task for training the model with this dataset. 

In [12]:
#call the above cell as a remote ray function
result=ray.get(train_fn.remote())

(train_fn pid=294) Downloading and preparing dataset imdb/plain_text to /home/ray/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0...


100%|██████████| 3/3 [00:00<00:00, 782.91it/s]                                                


(train_fn pid=294) Dataset imdb downloaded and prepared to /home/ray/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


Map: 100%|██████████| 50000/50000 [00:27<00:00, 1811.56 examples/s]
                                                                   


(train_fn pid=294) len of train Dataset({
(train_fn pid=294)     features: ['text', 'label', 'input_ids', 'attention_mask'],
(train_fn pid=294)     num_rows: 100
(train_fn pid=294) }) and test Dataset({
(train_fn pid=294)     features: ['text', 'label', 'input_ids', 'attention_mask'],
(train_fn pid=294)     num_rows: 100
(train_fn pid=294) })
(train_fn pid=294) huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
(train_fn pid=294) To disable this warning, you can either:
(train_fn pid=294) 	- Avoid using `tokenizers` before the fork if possible
(train_fn pid=294) 	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
(train_fn pid=294) == Status ==
(train_fn pid=294) Current time: 2023-04-16 12:36:17 (running for 00:00:04.89)
(train_fn pid=294) Memory usage on this node: 69.0/503.7 GiB 
(train_fn pid=294) Using FIFO scheduling algorithm.
(train_fn pid=294) Resources r

...

(train_fn pid=294) 2023-04-16 12:49:05,848	WARNING util.py:244 -- The `process_trial_save` operation took 3.456 s, which may be a performance bottleneck.


(train_fn pid=294) Result for HuggingFaceTrainer_f1d17_00000:
(train_fn pid=294)   _time_this_iter_s: 31.07945704460144
(train_fn pid=294)   _timestamp: 1681674542
(train_fn pid=294)   _training_iteration: 2
(train_fn pid=294)   date: 2023-04-16_12-49-02
(train_fn pid=294)   done: true
(train_fn pid=294)   epoch: 1.0
(train_fn pid=294)   experiment_id: c144d505d0c149efa2a1c3c5a0f5bdc2
(train_fn pid=294)   experiment_tag: '0'
(train_fn pid=294)   hostname: hfgputest-worker-small-group-hfgputest-6wlvr
(train_fn pid=294)   iterations_since_restore: 2
(train_fn pid=294)   learning_rate: 0.0
(train_fn pid=294)   loss: 0.1855
(train_fn pid=294)   node_ip: 10.130.0.183
(train_fn pid=294)   pid: 117
(train_fn pid=294)   should_checkpoint: true
(train_fn pid=294)   step: 521
(train_fn pid=294)   time_since_restore: 765.0372877120972
(train_fn pid=294)   time_this_iter_s: 31.022125244140625
(train_fn pid=294)   time_total_s: 765.0372877120972
(train_fn pid=294)   timestamp: 1681674542
(train_fn 

(train_fn pid=294) 2023-04-16 12:49:09,870	WARNING util.py:244 -- The `process_trial_save` operation took 1.875 s, which may be a performance bottleneck.


(train_fn pid=294) == Status ==
(train_fn pid=294) Current time: 2023-04-16 12:49:11 (running for 00:12:59.18)
(train_fn pid=294) Memory usage on this node: 70.6/503.7 GiB 
(train_fn pid=294) Using FIFO scheduling algorithm.
(train_fn pid=294) Resources requested: 0/26 CPUs, 0/3 GPUs, 0.0/52.15 GiB heap, 0.0/15.46 GiB objects
(train_fn pid=294) Result logdir: /home/ray/ray_results/HuggingFaceTrainer_2023-04-16_12-36-12
(train_fn pid=294) Number of trials: 1/1 (1 TERMINATED)
(train_fn pid=294) +--------------------------------+------------+------------------+--------+------------------+--------+-----------------+---------+
(train_fn pid=294) | Trial name                     | status     | loc              |   iter |   total time (s) |   loss |   learning_rate |   epoch |
(train_fn pid=294) |--------------------------------+------------+------------------+--------+------------------+--------+-----------------+---------|
(train_fn pid=294) | HuggingFaceTrainer_f1d17_00000 | TERMINATED | 1

(train_fn pid=294) 2023-04-16 12:49:11,740	INFO tune.py:777 -- Total run time: 779.38 seconds (779.18 seconds for the tuning loop).


In [13]:
from ray.train.torch import TorchCheckpoint
checkpoint: TorchCheckpoint = result
path = checkpoint.to_directory()

In [14]:
print(path)
!ls {path}

/tmp/checkpoint_tmp_41cdabea109d46b1bde91192b77a3ecf
E0416 19:49:19.333066640     978 completion_queue.cc:738]              Kick failed: UNKNOWN:Bad file descriptor {created_time:"2023-04-16T19:49:19.332851369+00:00", errno:9, os_error:"Bad file descriptor", syscall:"eventfd_write"}
config.json		_preprocessor	   trainer_state.json
_current_checkpoint_id	pytorch_model.bin  training_args.bin
_metadata.meta.pkl	rng_state_0.pth
optimizer.pt		scheduler.pt


In [ ]:
#log_dir=result.log_dir
#print(f"log_dir: {log_dir}")

# Inference using the checkpoint

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
#DistilbertTokenizerFast
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}
model = AutoModelForSequenceClassification.from_pretrained(path,num_labels=2, id2label=id2label, label2id=label2id)
text1 = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."
text2 = "This is a catastrophe. Each of the three movies had different actors that made it difficult to follow."
#inputs = tokenizer(text, return_tensors="pt")
batch=[text1,text2]
inputs = tokenizer(batch, padding=True, truncation=True, max_length=512, return_tensors="pt")

with torch.no_grad(): logits = model(**inputs).logits # For pytorch you have to unpack

/opt/app-root/lib64/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
print(logits)
print(torch.nn.Softmax(dim=1)(logits)) #tf.math.softmax(logits, axis=-1)

tensor([[-1.6680,  2.4469],
        [ 1.0801, -0.9055]])
tensor([[0.0161, 0.9839],
        [0.8793, 0.1207]])


In [17]:
import numpy as np
print(np.array(logits))
predicted_class_id = np.array(logits).argmax(axis=1)
print(predicted_class_id)
print([model.config.id2label[i] for i in predicted_class_id])

[[-1.6679788   2.4469006 ]
 [ 1.0801115  -0.90551084]]
[1 0]
['POSITIVE', 'NEGATIVE']


# Convert to onyx

In [18]:
torch.onnx.export(
    model, 
    tuple(inputs.values()),
    f="torch-model.onnx",  
    input_names=['input_ids', 'attention_mask'], 
    output_names=['logits'], 
    dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence'}, 
                  'attention_mask': {0: 'batch_size', 1: 'sequence'}, 
                  'logits': {0: 'batch_size', 1: 'sequence'}}, 
    do_constant_folding=True, 
    opset_version=13, 
)

/opt/app-root/lib64/python3.8/site-packages/transformers/models/distilbert/modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


======= Diagnostic Run torch.onnx.export version 2.1.0.dev20230414+cu117 =======
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [19]:
from datasets import load_dataset
dataset = load_dataset("imdb")

Dataset imdb downloaded and prepared to /opt/app-root/src/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 705.36it/s]


In [20]:
import onnx
import onnxruntime
import torch
import numpy as np

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [22]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print(tokenizer)

session = onnxruntime.InferenceSession('torch-model.onnx', None)
text="This is a catastrophe."
inputs = tokenizer(text, return_tensors="np")
print(inputs)

result1 = session.run([i.name for i in session.get_outputs()], dict(inputs))
print(result1)

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}
predicted_class_id = np.array(result1).argmax().item()
print(id2label[predicted_class_id])

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)
{'input_ids': array([[  101,  2023,  2003,  1037, 25539,  1012,   102]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1]])}
[array([[ 0.46370822, -0.31326616]], dtype=float32)]
NEGATIVE


In [23]:
#import tensorflow as tf
#predictions = tf.math.softmax(result, axis=-1)
print(torch.nn.Softmax(dim=1)(torch.tensor(result1[0])))

tensor([[0.6850, 0.3150]])


In [24]:
text1 = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."
text2 = "This is a catastrophe."
batch=[text1,text2]
inputs = tokenizer(batch, padding=True, truncation=True, max_length=512, return_tensors="np")
result2 = session.run([i.name for i in session.get_outputs()], dict(inputs))
print(result2)
torch.nn.Softmax(dim=1)(torch.tensor(result2[0]))
print(np.argmax(torch.nn.Softmax(dim=1)(torch.tensor(result2[0])),axis=1))
print([id2label[i.item()] for i in torch.argmax(torch.nn.Softmax(dim=1)(torch.tensor(result2[0])),axis=1)])
labels=[id2label[labelid] for labelid in torch.argmax(torch.nn.Softmax(dim=1)(torch.tensor(result2[0])),axis=1).tolist()]
print(labels)

[array([[-1.6679785 ,  2.4469004 ],
       [ 0.46370822, -0.31326616]], dtype=float32)]
tensor([1, 0])
['POSITIVE', 'NEGATIVE']
['POSITIVE', 'NEGATIVE']


# Upload the model to S3 Bucket

In [26]:
import os
import boto3
from boto3 import session

key_id = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
session = boto3.session.Session(aws_access_key_id=key_id, aws_secret_access_key=secret_key)
s3_client = boto3.client('s3', aws_access_key_id=key_id, aws_secret_access_key=secret_key,endpoint_url=endpoint_url,verify=False)
buckets=s3_client.list_buckets()
for bucket in buckets['Buckets']: print(bucket['Name'])

spark-demo-07ae575f-1b38-4064-b358-c0f7e1a88ddb


/opt/app-root/lib64/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.openshift-storage.svc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [27]:
print(bucket['Name'])
modelfile='torch-model.onnx'
s3_client.upload_file(modelfile, bucket['Name'],'hf_model.onnx')

spark-demo-07ae575f-1b38-4064-b358-c0f7e1a88ddb


/opt/app-root/lib64/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.openshift-storage.svc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/app-root/lib64/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.openshift-storage.svc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/app-root/lib64/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.openshift-storage.svc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/app-root/lib64/python3.

In [28]:
[item.get("Key") for item in s3_client.list_objects_v2(Bucket=bucket['Name']).get("Contents")]

/opt/app-root/lib64/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.openshift-storage.svc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


['hf_model.onnx',
 'iris/model.joblib',
 'mnist-model/1/saved_model.pb',
 'mnist-model/1/variables/variables.data-00000-of-00001',
 'mnist-model/1/variables/variables.index',
 'mnist-svm.joblib',
 'mnist.onnx',
 'mnist_model_from_saved_model.onnx',
 'model.joblib',
 'saved_model.pb',
 'yolov5n.onnx']

Now manually deploy the model from Data Science Projects

---
# Submit inferencing request to Deployed model

In [ ]:
import requests
import json
URL='http://modelmesh-serving.huggingface.svc.cluster.local:8008/v2/models/hfmodel/infer' # underscore characters are removed
headers = {}
payload = {
        "inputs": [{ "name": "input_ids", "shape": inputs.get('input_ids').shape, "datatype": "INT64", "data": inputs.get('input_ids').tolist()},{ "name": "attention_mask", "shape": inputs.get('attention_mask').shape, "datatype": "INT64", "data": inputs.get('attention_mask').tolist()}]
    }

headers = {"content-type": "application/json"}
res = requests.post(URL, json=payload, headers=headers)
print(res)
print(res.text)

In [ ]:
result=[np.array(res.json().get('outputs')[0].get('data')).reshape(res.json().get('outputs')[0].get('shape'))]

In [ ]:
torch.nn.Softmax(dim=1)(torch.tensor(result[0]))
print(np.argmax(torch.nn.Softmax(dim=1)(torch.tensor(result[0])),axis=1))
print('Using item',[id2label[i.item()] for i in torch.argmax(torch.nn.Softmax(dim=1)(torch.tensor(result[0])),axis=1)])
labels=[id2label[labelid] for labelid in torch.argmax(torch.nn.Softmax(dim=1)(torch.tensor(result[0])),axis=1).tolist()]
print('Using to_list',labels)

Finally, we bring our resource cluster down and release/terminate the associated resources, bringing everything back to the way it was before our cluster was brought up.

# Conclusion
As shown in the above example, you can easily run your Huggingface transfer learning tasks easily and natively on CodeFlare. You can scale them from 1 to n GPUs without requiring you to make any significant code changes and leveraging the native Huggingface trainer. 

Also refer to additional notebooks that showcase other use cases
In our next notebook [./02_codeflare_workflows_encoding.ipynb ] shows an sklearn example and how you can leverage workflows to run experiment pipelines and explore multiple pipelines in parallel on CodeFlare cluster. 


In [ ]:
cluster.down()

In [ ]:
auth.logout()